In [6]:
# Edit tickers here (first cell on purpose)
TICKERS = [
    "AAPL",
    "MSFT",
    "NVDA",
]

# Optional: set True to ignore TICKERS and load from config/universe.toml
USE_UNIVERSE_TOML = False


In [7]:
from pathlib import Path
import sys

def find_repo_root(start: Path | None = None) -> Path:
    start = (start or Path.cwd()).resolve()
    for p in (start, *start.parents):
        if (p / "pyproject.toml").exists() or (p / ".git").exists():
            return p
    return start

REPO_ROOT = find_repo_root()
SRC = REPO_ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print("REPO_ROOT:", REPO_ROOT)
print("SRC:", SRC)


REPO_ROOT: C:\Users\MaartenEnde\Repos\scanner
SRC: C:\Users\MaartenEnde\Repos\scanner\src


In [8]:
# Params (defaults come from ingest_config.py)
from scanner_bot.config.ingest_config import (
    DB_PATH,
    INTERVAL,
    PERIOD,
    BATCH_SIZE,
    SLEEP_BETWEEN_BATCHES_SEC,
    UNIVERSE_TOML,
)
from scanner_bot.config.universe import load_universe_from_toml

# IMPORTANT: this must match your normalized DB 'sources.name'
SOURCE = "yfinance"

tickers = load_universe_from_toml(UNIVERSE_TOML) if USE_UNIVERSE_TOML else TICKERS
tickers = [t.strip().upper() for t in tickers if t and t.strip()]
len(tickers), tickers[:10], DB_PATH


(3,
 ['AAPL', 'MSFT', 'NVDA'],
 WindowsPath('C:/Users/MaartenEnde/Repos/scanner/data/db/scanner.db'))

In [9]:
from scanner_bot.infra.ingest.yfinance_ingest import ingest_yfinance_tickers

stats = ingest_yfinance_tickers(
    tickers=tickers,
    db_path=DB_PATH,
    source=SOURCE,
    interval_code=INTERVAL,
    period=PERIOD,
    batch_size=BATCH_SIZE,
    sleep_between_batches_sec=SLEEP_BETWEEN_BATCHES_SEC,
)
stats


[ingest_yf] db=C:\Users\MaartenEnde\Repos\scanner\data\db\scanner.db source=yfinance interval=1d period=max tickers=3
[ingest_yf] upserted AAPL: 11361 bars
[ingest_yf] upserted MSFT: 10035 bars
[ingest_yf] upserted NVDA: 6784 bars
[ingest_yf] done {'tickers_total': 3, 'ok': 3, 'empty': 0, 'total_bars': 28180, 'interval': '1d', 'period': 'max', 'source': 'yfinance', 'db_path': 'C:\\Users\\MaartenEnde\\Repos\\scanner\\data\\db\\scanner.db'}


{'tickers_total': 3,
 'ok': 3,
 'empty': 0,
 'total_bars': 28180,
 'interval': '1d',
 'period': 'max',
 'source': 'yfinance',
 'db_path': 'C:\\Users\\MaartenEnde\\Repos\\scanner\\data\\db\\scanner.db'}

In [10]:
# Sanity check: show most recent rows
from scanner_bot.infra.db.sqlite import SQLiteDB

db = SQLiteDB(DB_PATH, create_if_not_exists=False)
with db.connect() as c:
    rows = c.execute(
        "SELECT source, ticker, interval, ts_utc_ms FROM v_bars ORDER BY ts_utc_ms DESC LIMIT 10"
    ).fetchall()

[(r["source"], r["ticker"], r["interval"], r["ts_utc_ms"]) for r in rows]


[('yfinance', 'GOOGL', '1d', 1767916800000),
 ('yfinance', 'AMZN', '1d', 1767916800000),
 ('yfinance', 'AMGN', '1d', 1767916800000),
 ('yfinance', 'AAPL', '1d', 1767916800000),
 ('yfinance', 'AMAT', '1d', 1767916800000),
 ('yfinance', 'ASML', '1d', 1767916800000),
 ('yfinance', 'BIIB', '1d', 1767916800000),
 ('yfinance', 'AVGO', '1d', 1767916800000),
 ('yfinance', 'CSCO', '1d', 1767916800000),
 ('yfinance', 'CSX', '1d', 1767916800000)]